In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [81]:
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import random
import re
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer

# Ensure required nltk data is downloaded
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

class CommentGenerator:
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

    def analyze_sentiment(self, text):
        """
        Analyzes the sentiment of the given text.
        Returns:
            - Sentiment: ('positive', 'neutral', 'negative')
        """
        sentiment_scores = self.analyzer.polarity_scores(text)
        sentiment = sentiment_scores['compound']

        if sentiment >= 0.05:
            return 'positive'
        elif sentiment <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    def analyze_tone(self, text):
        """
        Analyzes the tone of the given text.
        Returns:
            - Tone: ('friendly', 'funny', 'congratulating', 'questioning', 'disagreement')
        Using keyword detection"""
        positive_keywords = ['great', 'fantastic', 'excellent', 'good', 'amazing', 'wonderful']
        funny_keywords = ['haha', 'lol', 'funny', 'hilarious', 'joke', 'humor']
        congratulating_keywords = ['congratulations', 'kudos', 'well done', 'bravo', 'praise']
        questioning_keywords = ['why', 'how', 'what', 'question', 'curious', 'wonder']
        disagreement_keywords = ['disagree', 'wrong', 'not', 'no', 'incorrect', 'however']

        text_lower = text.lower()

        def contains_keywords(keywords):
            return any(re.search(r'\b' + keyword + r'\b', text_lower) for keyword in keywords)

        if contains_keywords(positive_keywords):
            return 'friendly'
        elif contains_keywords(funny_keywords):
            return 'funny'
        elif contains_keywords(congratulating_keywords):
            return 'congratulating'
        elif contains_keywords(questioning_keywords):
            return 'questioning'
        elif contains_keywords(disagreement_keywords):
            return 'disagreement'
        else:
            # Fallback to sentiment-based tone if no specific keywords are found
            sentiment = self.analyze_sentiment(text)
            if sentiment == 'positive':
                return random.choice(['friendly', 'congratulating'])
            elif sentiment == 'negative':
                return 'disagreement'
            else:
                return random.choice(['funny', 'questioning'])

    def extract_entities(self, text):
        """
        Extracts named entities from the text.
        Returns:
            - entities: List of named entities
        Using Spacy's NER capabibilities"""
        doc = self.nlp(text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "EVENT", "PRODUCT", "WORK_OF_ART", "LANGUAGE"]]
        return entities

    def summarize_text(self, text):
        """
        Summarizes the given text.
        Returns:
            - summary: Summarized text
        """
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, sentences_count=2)
        return " ".join(str(sentence) for sentence in summary)

    def generate_comments(self, text):
        """
        Generates five different types of comments based on the given text.
        Returns:
            - comments: A dictionary containing five types of comments
        """
        tone = self.analyze_tone(text)
        sentiment = self.analyze_sentiment(text)
        entities = self.extract_entities(text)
        summary = self.summarize_text(text)

        # Predefined comments with placeholders for entities
        friendly_comments = [
            "Great job on covering {}! This article is really insightful and well-written.",
            "I loved reading this about {}. Keep up the good work!",
            "This is fantastic! Thanks for sharing about {}."
        ]

        funny_comments = [
            "Haha, the part about {} made my day! Can't stop laughing.",
            "This is hilarious! Well done on {}!",
            "Loved the humor in this piece about {}!"
        ]

        congratulating_comments = [
            "Congratulations on writing such an engaging piece about {}!",
            "Well done! This deserves praise, especially about {}.",
            "Kudos for this excellent article about {}!"
        ]

        questioning_comments = [
            "I'm curious about something in this article regarding {}: [Insert question here]",
            "Could you elaborate more on this point about {}?",
            "I have a question regarding your argument on {}."
        ]

        disagreement_comments = [
            "I respectfully disagree with some points about {} in this article. Let's discuss further.",
            "I have a different perspective on this topic, especially about {}.",
            "Interesting article, but I don't fully agree with some of the points made about {}."
        ]

        # Select a random comment and insert entities if available
        entity_str = ", ".join(entities) if entities else "the topic"
        comments = {
            'friendly': random.choice(friendly_comments).format(entity_str),
            'funny': random.choice(funny_comments).format(entity_str),
            'congratulating': random.choice(congratulating_comments).format(entity_str),
            'questioning': random.choice(questioning_comments).format(entity_str),
            'disagreement': random.choice(disagreement_comments).format(entity_str)
        }

        return comments

# Example usage:
if __name__ == "__main__":
    generator = CommentGenerator()
    article = input("Enter the article: ")
    tone = generator.analyze_tone(article)
    sentiment = generator.analyze_sentiment(article)
    print("Tone:", tone)
    print("Sentiment:", sentiment)
    comments = generator.generate_comments(article)
    print("Generated Comments:")
    for comment_type, comment in comments.items():
        print(f"{comment_type.capitalize()}: {comment}")


Enter the article: You get one, precious life. How do you decide the best way to spend your time? Productivity gurus will often suggest that you focus on being effective rather than being efficient.  Efficiency is about getting more things done. Effectiveness is about getting the right things done. Peter Drucker, the well-known management consultant, once encapsulated the idea by writing, “There is nothing so useless as doing efficiently that which should not be done at all.”  In other words, making progress is not just about being productive. It’s about being productive on the right things.  But how do you decide what the “right things” are? One of the most trusted approaches is to use the Pareto Principle, which is more commonly known as the 80/20 Rule.  The 80/20 Rule states that, in any particular domain, a small number of things account for the majority of the results. For example, 80 percent of the land in Italy is owned by 20 percent of the people. Or, 75 percent of NBA champion

In [80]:
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import random
import re
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer

# Ensure required nltk data is downloaded
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

class CommentGenerator:
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

    def analyze_sentiment(self, text):
        """
        Analyzes the sentiment of the given text.
        Returns:
            - Sentiment: ('positive', 'neutral', 'negative')
        """
        sentiment_scores = self.analyzer.polarity_scores(text)
        sentiment = sentiment_scores['compound']

        if sentiment >= 0.05:
            return 'positive'
        elif sentiment <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    def analyze_tone(self, text):
        """
        Analyzes the tone of the given text.
        Returns:
            - Tone: ('friendly', 'funny', 'congratulating', 'questioning', 'disagreement')
        """
        positive_keywords = ['great', 'fantastic', 'excellent', 'good', 'amazing', 'wonderful']
        funny_keywords = ['haha', 'lol', 'funny', 'hilarious', 'joke', 'humor']
        congratulating_keywords = ['congratulations', 'kudos', 'well done', 'bravo', 'praise']
        questioning_keywords = ['why', 'how', 'what', 'question', 'curious', 'wonder']
        disagreement_keywords = ['disagree', 'wrong', 'not', 'no', 'incorrect', 'however']

        text_lower = text.lower()

        def contains_keywords(keywords):
            return any(re.search(r'\b' + keyword + r'\b', text_lower) for keyword in keywords)

        if contains_keywords(positive_keywords):
            return 'friendly'
        elif contains_keywords(funny_keywords):
            return 'funny'
        elif contains_keywords(congratulating_keywords):
            return 'congratulating'
        elif contains_keywords(questioning_keywords):
            return 'questioning'
        elif contains_keywords(disagreement_keywords):
            return 'disagreement'
        else:
            # Fallback to sentiment-based tone if no specific keywords are found
            sentiment = self.analyze_sentiment(text)
            if sentiment == 'positive':
                return random.choice(['friendly', 'congratulating'])
            elif sentiment == 'negative':
                return 'disagreement'
            else:
                return random.choice(['funny', 'questioning'])

    def extract_entities(self, text):
        """
        Extracts named entities from the text.
        Returns:
            - entities: List of named entities
        """
        doc = self.nlp(text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "EVENT", "PRODUCT", "WORK_OF_ART", "LANGUAGE"]]
        return entities

    def summarize_text(self, text):
        """
        Summarizes the given text.
        Returns:
            - summary: Summarized text
        """
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, sentences_count=2)
        return " ".join(str(sentence) for sentence in summary)

    def generate_comments(self, text):
        """
        Generates five different types of comments based on the given text.
        Returns:
            - comments: A dictionary containing five types of comments
        """
        tone = self.analyze_tone(text)
        sentiment = self.analyze_sentiment(text)
        entities = self.extract_entities(text)
        summary = self.summarize_text(text)

        # Predefined comments with placeholders for entities
        friendly_comments = [
            "Great job on covering {}! This article is really insightful and well-written.",
            "I loved reading this about {}. Keep up the good work!",
            "This is fantastic! Thanks for sharing about {}."
        ]

        funny_comments = [
            "Haha, the part about {} made my day! Can't stop laughing.",
            "This is hilarious! Well done on {}!",
            "Loved the humor in this piece about {}!"
        ]

        congratulating_comments = [
            "Congratulations on writing such an engaging piece about {}!",
            "Well done! This deserves praise, especially about {}.",
            "Kudos for this excellent article about {}!"
        ]

        questioning_comments = [
            "I'm curious about something in this article regarding {}: [Insert question here]",
            "Could you elaborate more on this point about {}?",
            "I have a question regarding your argument on {}."
        ]

        disagreement_comments = [
            "I respectfully disagree with some points about {} in this article. Let's discuss further.",
            "I have a different perspective on this topic, especially about {}.",
            "Interesting article, but I don't fully agree with some of the points made about {}."
        ]

        # Select a random comment and insert entities if available
        entity_str = ", ".join(entities) if entities else "the topic"
        comments = {
            'friendly': random.choice(friendly_comments).format(entity_str),
            'funny': random.choice(funny_comments).format(entity_str),
            'congratulating': random.choice(congratulating_comments).format(entity_str),
            'questioning': random.choice(questioning_comments).format(entity_str),
            'disagreement': random.choice(disagreement_comments).format(entity_str)
        }

        return comments

# Example usage:
if __name__ == "__main__":
    generator = CommentGenerator()
    article = input("Enter the article: ")
    tone = generator.analyze_tone(article)
    sentiment = generator.analyze_sentiment(article)
    print("Tone:", tone)
    print("Sentiment:", sentiment)
    comments = generator.generate_comments(article)
    print("Generated Comments:")
    for comment_type, comment in comments.items():
        print(f"{comment_type.capitalize()}: {comment}")


Enter the article: When my friend Beck Tench began her weight loss journey, she repeatedly asked herself the question, “What would a healthy person do?”  When she was deciding what to order a restaurant: what would a healthy person order? When she was sitting around on a Saturday morning: what would a healthy person do with that time? Beck didn’t feel like a healthy person at the start, but she figured that if she acted like a healthy person, then eventually she would become one. And within a few years, she had lost over 100 pounds.  Confidence is a wonderful thing to have, but if you find yourself overcome with fear, self-doubt, or uncertainty, then let your behavior drive your beliefs. Play as if you’re at your best. Work as if you’re on top of your game. Talk to that person as if you’re feeling confident. You can use bold actions to trigger a bold mindset.
Tone: friendly
Sentiment: positive
Generated Comments:
Friendly: I loved reading this about Beck Tench. Keep up the good work!
F

In [79]:
import nltk
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import random
import re
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer

# Ensure required nltk data is downloaded
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

class CommentGenerator:
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
        self.nlp = spacy.load("en_core_web_sm")

    def analyze_sentiment(self, text):
        """
        Analyzes the sentiment of the given text.
        Returns:
            - Sentiment: ('positive', 'neutral', 'negative')
        """
        sentiment_scores = self.analyzer.polarity_scores(text)
        sentiment = sentiment_scores['compound']

        if sentiment >= 0.05:
            return 'positive'
        elif sentiment <= -0.05:
            return 'negative'
        else:
            return 'neutral'

    def analyze_tone(self, text):
        """
        Analyzes the tone of the given text.
        Returns:
            - Tone: ('friendly', 'funny', 'congratulating', 'questioning', 'disagreement')
        """
        positive_keywords = ['great', 'fantastic', 'excellent', 'good', 'amazing', 'wonderful']
        funny_keywords = ['haha', 'lol', 'funny', 'hilarious', 'joke', 'humor']
        congratulating_keywords = ['congratulations', 'kudos', 'well done', 'bravo', 'praise']
        questioning_keywords = ['why', 'how', 'what', 'question', 'curious', 'wonder']
        disagreement_keywords = ['disagree', 'wrong', 'not', 'no', 'incorrect', 'however']

        text_lower = text.lower()

        def contains_keywords(keywords):
            return any(re.search(r'\b' + keyword + r'\b', text_lower) for keyword in keywords)

        if contains_keywords(positive_keywords):
            return 'friendly'
        elif contains_keywords(funny_keywords):
            return 'funny'
        elif contains_keywords(congratulating_keywords):
            return 'congratulating'
        elif contains_keywords(questioning_keywords):
            return 'questioning'
        elif contains_keywords(disagreement_keywords):
            return 'disagreement'
        else:
            # Fallback to sentiment-based tone if no specific keywords are found
            sentiment = self.analyze_sentiment(text)
            if sentiment == 'positive':
                return random.choice(['friendly', 'congratulating'])
            elif sentiment == 'negative':
                return 'disagreement'
            else:
                return random.choice(['funny', 'questioning'])

    def extract_entities(self, text):
        """
        Extracts named entities from the text.
        Returns:
            - entities: List of named entities
        """
        doc = self.nlp(text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE", "EVENT", "PRODUCT", "WORK_OF_ART", "LANGUAGE"]]
        return entities

    def summarize_text(self, text):
        """
        Summarizes the given text.
        Returns:
            - summary: Summarized text
        """
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, sentences_count=2)
        return " ".join(str(sentence) for sentence in summary)

    def generate_comments(self, text):
        """
        Generates five different types of comments based on the given text.
        Returns:
            - comments: A dictionary containing five types of comments
        """
        tone = self.analyze_tone(text)
        sentiment = self.analyze_sentiment(text)
        entities = self.extract_entities(text)
        summary = self.summarize_text(text)

        # Predefined comments with placeholders for entities
        friendly_comments = [
            "Great job on covering {}! This article is really insightful and well-written.",
            "I loved reading this about {}. Keep up the good work!",
            "This is fantastic! Thanks for sharing about {}."
        ]

        funny_comments = [
            "Haha, the part about {} made my day! Can't stop laughing.",
            "This is hilarious! Well done on {}!",
            "Loved the humor in this piece about {}!"
        ]

        congratulating_comments = [
            "Congratulations on writing such an engaging piece about {}!",
            "Well done! This deserves praise, especially about {}.",
            "Kudos for this excellent article about {}!"
        ]

        questioning_comments = [
            "I'm curious about something in this article regarding {}: [Insert question here]",
            "Could you elaborate more on this point about {}?",
            "I have a question regarding your argument on {}."
        ]

        disagreement_comments = [
            "I respectfully disagree with some points about {} in this article. Let's discuss further.",
            "I have a different perspective on this topic, especially about {}.",
            "Interesting article, but I don't fully agree with some of the points made about {}."
        ]

        # Select a random comment and insert entities if available
        entity_str = ", ".join(entities) if entities else "the topic"
        comments = {
            'friendly': random.choice(friendly_comments).format(entity_str),
            'funny': random.choice(funny_comments).format(entity_str),
            'congratulating': random.choice(congratulating_comments).format(entity_str),
            'questioning': random.choice(questioning_comments).format(entity_str),
            'disagreement': random.choice(disagreement_comments).format(entity_str)
        }

        return comments

# Example usage:
if __name__ == "__main__":
    generator = CommentGenerator()
    article = input("Enter the article: ")
    tone = generator.analyze_tone(article)
    sentiment = generator.analyze_sentiment(article)
    print("Tone:", tone)
    print("Sentiment:", sentiment)
    comments = generator.generate_comments(article)
    print("Generated Comments:")
    for comment_type, comment in comments.items():
        print(f"{comment_type.capitalize()}: {comment}")


Enter the article: How can we apply the growth mindset to creativity in practical terms? In my experience, it comes down to one thing: the willingness to look bad when pursuing an activity.  As Dweck says, the growth mindset is focused more on the process than the outcome. This is easy to accept in theory, but very hard to stick to in practice. Most people don’t want to deal with the accompanying embarrassment or shame that is often required to learn a new skill.  The list of mistakes that you can never recover from is very short. I think most of us realize this on some level. We know that our lives will not be destroyed if that book we write doesn’t sell or if we get turned down by a potential date or if we forget someone’s name when we introduce them. It’s not necessarily what comes after the event that worries us. It’s the possibility of looking stupid, feeling humiliated, or dealing with embarrassment along the way that prevents us from getting started at all.  In order to fully em